In [88]:
import doctest
import csv
import re
import numpy as np
import collections

from math import gcd
from itertools import repeat
from functools import reduce

In [89]:
with open('eng.csv') as eng_f:
    eng_freq = {k.lower(): float(v) for k, v in csv.reader(eng_f)}
eng_freq

{'a': 0.08167,
 'b': 0.01492,
 'c': 0.02782,
 'd': 0.04253,
 'e': 0.12702,
 'f': 0.0228,
 'g': 0.02015,
 'h': 0.06094,
 'i': 0.06966,
 'j': 0.00153,
 'k': 0.00772,
 'l': 0.04025,
 'm': 0.02406,
 'n': 0.06749,
 'o': 0.07507,
 'p': 0.01929,
 'q': 0.00095,
 'r': 0.05987,
 's': 0.06327,
 't': 0.09056,
 'u': 0.02758,
 'v': 0.00978,
 'w': 0.0236,
 'x': 0.0015,
 'y': 0.01974,
 'z': 0.00074}

## Vigenere Cipher

In [90]:
def generate_key(initial_str: str, key_word: str):
    """
    Generate the key in a cyclic manner
    until it's length isn't equal to the length of original text

    >>> generate_key('CRYPTOGRAPHYANDDATASECURITY', 'MOUSE')
    'MOUSEMOUSEMOUSEMOUSEMOUSEMO'
    """
    key_word_len = len(key_word)
    initial_string_len = len(initial_str)

    generated_list = list(repeat(key_word, initial_string_len // key_word_len + 1))

    return "".join(generated_list)[:initial_string_len]


def encrypt(initial_str, key_str):
    """
    Return the encrypted text generated with the help of the key

    >>> encrypt('CRYPTOGRAPHYANDDATASECURITY', 'MOUSEMOUSEMOUSEMOUSEMOUSEMO')
    'OFSHXAULSTTMUFHPONSWQQOJMFM'
    """
    encrypted_text = [
        chr((ord(initial_str[i]) + ord(key_str[i])) % 26 + ord('A'))
        for i in range(len(initial_str))
    ]
    return "".join(encrypted_text)


def decrypt(encrypted_str, key_str):
    """
    Decrypt the encrypted text and returns the original text

    >>> decrypt('OFSHXAULSTTMUFHPONSWQQOJMFM', 'MOUSEMOUSEMOUSEMOUSEMOUSEMO')
    'CRYPTOGRAPHYANDDATASECURITY'
    """
    decrypted_text = [
        chr((ord(encrypted_str[i]) - ord(key_str[i]) + 26) % 26 + ord('A'))
        for i in range(len(encrypted_str))
    ]
    return "".join(decrypted_text)

## Kasiski_test

In [91]:
def search_l_gramms(cipher_text, key_len):
    """
    Search all repeated key_len gramms and their spacing 
    
    >>> search_l_gramms("bla bla asd bla asd xx yy zz", 3)
    {'bla': array([4, 8]), 'asd': array([8])}
    """
    gramms = {}
    for i in range(len(cipher_text) - key_len):
        sub_str = cipher_text[i: i+key_len]

        if sub_str.isalpha() and sub_str not in gramms:
            matches = [
                m.start()
                for m in re.finditer(f'(?={sub_str})', cipher_text)
            ]
            if matches:
                gramms[sub_str] = np.diff(matches)
        
    return gramms

In [92]:
def get_key_len(gramms: dict):
    """
    Return gcd of repeated strings distances 
    if strings count bigger than 3
    >>> get_key_len({'a': [4, 8, 16], 'b': [2, 4], 'c': [8, 16, 48]})
    4
    """
    return reduce(
        gcd,
        [
            reduce(gcd, len_diff_list)
            for len_diff_list in gramms.values()
            if len(len_diff_list) > 2
        ]
    )

In [93]:
doctest.testmod()

TestResults(failed=0, attempted=8)

In [94]:
def get_most_frequently_encountered_letters(text: str, key_len: int) -> list:
    """
    Return the most common letters string in substrings separated by key length
    
    >>> get_most_frequently_encountered_letters('blablabbb', 3)
    'bla'
    
    >>> get_most_frequently_encountered_letters('blablabbb', 2)
    'bb'
    """
    count_res = [
        collections.Counter(text[i::key_len]).most_common(1)
        for i in range(key_len)
    ]
    # count_res = [[('b', 4)], [('l', 3)], [('a', 5)]]
    return [_[0][0] for _ in count_res]

In [95]:
def build_key(letter_list):
    """
    #TODO
    """
    return "".join(
        [ 
            chr(
                (ord('A') + ord(letter.upper()) - ord('E') + 26) % 26
            ) 
            for letter in letter_list
        ]
    )